# climpy — AMO, Tripole, Corelații, Regresii, CCA

**Date folosite:**

| Dataset | Sursă | Tip | Perioadă |
|---|---|---|---|
| SST ERSSTv5 | NOAA PSL | raw monthly means | 1854–2019 |
| SAT Berkeley Earth | Berkeley Earth | deja anomalii | 1979–2019 |
| SLP NCEP-R1 | NOAA PSL | raw monthly means (Pa) | 1979–2019 |
| Precip GPCP v2.3 | NOAA PSL | raw monthly means (mm/day) | 1979–2019 |

> **Notă stippling:** corelațiile/regresiile folosesc indicii **anuali** (ne-neteziți)  
> pentru a menține N_eff suficient de mare. Media mobilă 7 ani e folosită  
> doar pentru vizualizarea time series-urilor.

In [ ]:
import subprocess, sys, os

# 1. Instalare pachete (cartopy primul)
subprocess.run(['pip', 'install', '-q',
    'cartopy', 'eofs', 'cmocean', 'cftime',
    'xarray', 'scipy', 'netCDF4'], check=True)

# 2. Clonează climpy de pe GitHub
if os.path.exists('/content/climpy'):
    subprocess.run(['rm', '-rf', '/content/climpy'])
subprocess.run(['git', 'clone',
    'https://github.com/CostiD/climpy.git',
    '/content/climpy'], check=True)
sys.path.insert(0, '/content/climpy')

import climpy
print(f'✓ climpy {climpy.__version__} gata')

In [ ]:
# ── Descărcare date ──────────────────────────────────────────────────────────
os.makedirs('data/real', exist_ok=True)
os.makedirs('figures',   exist_ok=True)

DATASETS = {
    'data/real/sst.nc': (
        'https://downloads.psl.noaa.gov/Datasets/noaa.ersst.v5/sst.mnmean.nc',
        'SST ERSSTv5  (raw, ~35 MB)'
    ),
    'data/real/sat_best.nc': (
        'https://berkeley-earth-temperature.s3.amazonaws.com/Global/Gridded/Land_and_Ocean_LatLong1.nc',
        'SAT Berkeley Earth (deja anomalii, ~180 MB)'
    ),
    'data/real/slp.nc': (
        'https://downloads.psl.noaa.gov/Datasets/ncep.reanalysis.derived/surface/slp.mon.mean.nc',
        'SLP NCEP-R1  (raw, ~35 MB)'
    ),
    'data/real/precip.nc': (
        'https://downloads.psl.noaa.gov/Datasets/gpcp/precip.mon.mean.nc',
        'Precip GPCP v2.3 (raw global, ~15 MB)'
    ),
}

for dest, (url, label) in DATASETS.items():
    if os.path.exists(dest):
        mb = os.path.getsize(dest) // 1024 // 1024
        print(f'  ✓ {label:45s} deja descărcat ({mb} MB)')
    else:
        print(f'  ↓ {label:45s} ...')
        ret = os.system(f'wget -q --show-progress -O {dest} "{url}"')
        if ret == 0:
            mb = os.path.getsize(dest) // 1024 // 1024
            print(f'    ✓ {mb} MB')
        else:
            print(f'    ✗ EROARE — verifică conexiunea')
print('\nGata.')

## 1 — Imports & stil

In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import cmocean.cm as cmo
import xarray as xr
import climpy
from climpy.analysis import correlation_pvalue, regression_pvalue
from climpy.analysis import CCA

%matplotlib inline
plt.rcParams['figure.dpi'] = 150
climpy.use_style('nature')

DATA = Path('data/real')

CMAP_SST    = cmo.balance
CMAP_SAT    = cmo.balance
CMAP_PRECIP = cmo.tarn
CMAP_SLP    = cmo.curl

# SST: perioada completă ERSSTv5 (1854–2019) — pentru EOF stabil
SST_PERIOD  = ('1854-01', '2019-12')
SST_REF     = ('1951-01', '1980-12')   # referință clasică pentru SST

# SAT/SLP/Precip: intersecție disponibilă (GPCP din 1979)
FIELD_PERIOD = ('1979-01', '2019-12')
FIELD_REF    = ('1981-01', '2010-12')  # referință WMO standard

print('✓ Gata')

## 2 — Preprocessing SST (1854–2019)

SST pe perioada completă → EOF mai stabil și reprezentativ fizic.  
Corelațiile/regresiile se aliniează automat cu câmpurile (1979–2019) prin `align_yr()`.

In [ ]:
sst_r = climpy.preprocess(
    DATA / 'sst.nc',
    var            = 'sst',
    fill_value     = -9.96921e+36,
    lon_convention = '[-180,180]',
    time           = SST_PERIOD,      # ← perioada completă ERSSTv5
    lat            = (0, 80),
    lon            = (-76, 20),
    ref_period     = SST_REF,
    compute_annual = True,
    subtract_gm    = True,
)
annual_sst = sst_r['annual_no_gm'].dropna('year', how='all')
print(f'SST: {dict(annual_sst.sizes)}')
print(f'Ani: {int(annual_sst.year[0])} – {int(annual_sst.year[-1])}')

## 3 — Preprocessing SAT, SLP, Precip (1979–2019)

In [ ]:
# SAT — Berkeley Earth (deja anomalii — NU se recalculează)
sat_da   = climpy.load_berkeley_earth(DATA / 'sat_best.nc',
               time=FIELD_PERIOD, lon_convention='[-180,180]')
sat_djfm = climpy.seasonal_means(sat_da, months=[12,1,2,3]).dropna('year', how='all')
sat_jjas = climpy.seasonal_means(sat_da, months=[6,7,8,9]).dropna('year', how='all')
print(f'SAT DJFM: {dict(sat_djfm.sizes)}')

# SLP — NCEP (raw Pa → anomalii calculate)
slp_r    = climpy.preprocess(DATA / 'slp.nc', var='slp',
               fill_value     = 9.96921e+36,   # ← fill value NCEP
               lon_convention = '[-180,180]',
               time=FIELD_PERIOD, ref_period=FIELD_REF,
               compute_annual=False, subtract_gm=False)
slp_djfm = climpy.seasonal_means(slp_r['anom'], months=[12,1,2,3]).dropna('year', how='all')
# NU împărți la 100 — NCEP slp.mon.mean.nc este deja în Pa dar anomaliile
# sezoniere sunt mici; verifică cu print de mai jos
print(f'SLP DJFM range: {float(slp_djfm.min()):.2f} – {float(slp_djfm.max()):.2f}')
print(f'SLP DJFM units (din atribute):', slp_r["anom"].attrs.get('units', 'necunoscut'))
# Dacă range e ±15000 → e în Pa → împarte la 100
# Dacă range e ±15   → e deja în hPa → nu împărți
import xarray as xr
_slp_raw = xr.open_dataset(DATA / 'slp.nc')
_mean_val = float(_slp_raw['slp'].isel(time=0).mean())
print(f'Valoare medie SLP brut (primul timestep): {_mean_val:.1f}')
print('→ dacă ~101000: Pa; dacă ~1010: deja hPa')
_slp_raw.close()
print(f'SLP DJFM: {dict(slp_djfm.sizes)}')

# Precip — GPCP (raw mm/day → anomalii calculate)
pr_r     = climpy.preprocess(DATA / 'precip.nc', var='precip',
               lon_convention='[-180,180]',
               time=FIELD_PERIOD, ref_period=FIELD_REF,
               compute_annual=False, subtract_gm=False)
pr_djfm  = climpy.seasonal_means(pr_r['anom'], months=[12,1,2,3]).dropna('year', how='all')
pr_jjas  = climpy.seasonal_means(pr_r['anom'], months=[6,7,8,9]).dropna('year', how='all')
print(f'Precip DJFM: {dict(pr_djfm.sizes)}')

## 4 — EOF SST → AMO + Tripole

EOF pe **1854–2019** (~165 ani) → pattern-uri fizic stabile.

In [ ]:
solver = climpy.EOF(annual_sst, min_variance=70)
solver.summary()
eofs = solver.eofs()
pcs  = solver.pcs()
frac = solver.variance_fraction()

In [ ]:
# ── Diagnostic EOF brut ──────────────────────────────────────────────────────
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point

fig, axes = plt.subplots(1, 2, figsize=(10, 3.5),
    subplot_kw={'projection': climpy.Map()})
for ax, mi, title in zip(axes, [0,1], ['EOF 1 (brut)','EOF 2 (brut)']):
    data = eofs.isel(mode=mi).values
    vm   = np.nanpercentile(np.abs(data), 98)
    dc, lc = add_cyclic_point(data, coord=eofs.lon.values)
    ax.set_extent([-80,25,-2,82], crs=climpy.Map())
    cf = ax.contourf(lc, eofs.lat.values, dc, levels=11, vmin=-vm, vmax=vm,
                     cmap=cmo.balance, transform=climpy.Map(), zorder=1)
    ax.add_feature(cfeature.LAND.with_scale('110m'), facecolor='#2E2E2E', zorder=3)
    ax.coastlines(linewidth=0.35, zorder=4)
    gl = ax.gridlines(draw_labels=True, linewidth=0.2, color='#CCCCCC',
                      linestyle='--', zorder=5)
    gl.top_labels = gl.right_labels = False
    gl.xlabel_style = gl.ylabel_style = {'size': 6}
    plt.colorbar(cf, ax=ax, orientation='horizontal', pad=0.04,
                 shrink=0.85, label='°C').ax.tick_params(labelsize=6)
    ax.set_title(f'  {title}  ({float(frac[mi]):.1f}%)', loc='left', fontsize=8)
plt.suptitle('Diagnostic — ajustează a, b în celula următoare',
             fontsize=8, style='italic', color='#666')
plt.tight_layout()
plt.show()

print(f'EOF1  N(>50N)={float(eofs.isel(mode=0).sel(lat=slice(50,80)).mean()):.3f}',
      f' S(<30N)={float(eofs.isel(mode=0).sel(lat=slice(0,30)).mean()):.3f}')
print(f'EOF2  N(>50N)={float(eofs.isel(mode=1).sel(lat=slice(50,80)).mean()):.3f}',
      f' S(<30N)={float(eofs.isel(mode=1).sel(lat=slice(0,30)).mean()):.3f}')

In [ ]:
# ── Combinare liniară → AMO + Tripole ───────────────────────────────────────
# Ghid: AMO = pattern UNIFORM; Tripole = 3 CENTRE ALTERNANTE Nord/Central/Sud
# Dacă EOF1≈AMO, EOF2≈Tripole: a_amo=1, b_amo=1 și a_tripol=1, b_tripol=-1
# Dacă semnul e invers față de convenție (AMO warm = albastru): a=-1

a_amo, b_amo       =  1,  1    # ← ajustează după diagnostic
a_tripol, b_tripol =  1, -1    # ← ajustează după diagnostic

eof_amo    = climpy.lincomb(eofs.isel(mode=0), eofs.isel(mode=1), a=a_amo,    b=b_amo)
eof_tripol = climpy.lincomb(eofs.isel(mode=0), eofs.isel(mode=1), a=a_tripol, b=b_tripol)
pc_amo     = climpy.lincomb(pcs.isel(mode=0),  pcs.isel(mode=1),  a=a_amo,    b=b_amo)
pc_tripol  = climpy.lincomb(pcs.isel(mode=0),  pcs.isel(mode=1),  a=a_tripol, b=b_tripol)

# MA doar pentru vizualizare time series
pc_amo_ma7    = climpy.moving_average(pc_amo,    n=7)
pc_tripol_ma7 = climpy.moving_average(pc_tripol, n=7)

def pc_to_year(pc):
    """PC cu dim 'time' (datetime fake) → DataArray cu dim 'year' (int)."""
    if 'time' in pc.dims:
        yrs = pc.time.dt.year.values
        return (pc.assign_coords(year=('time', yrs))
                  .swap_dims({'time': 'year'})
                  .drop_vars('time'))
    return pc

# Indici anuali (pentru corelații/regresii — N_eff mai mare)
pc_amo_yr     = pc_to_year(pc_amo)
pc_tripol_yr  = pc_to_year(pc_tripol)

# Indici MA (doar pentru time series plots)
pc_amo_ma7_yr    = pc_to_year(pc_amo_ma7)
pc_tripol_ma7_yr = pc_to_year(pc_tripol_ma7)

print(f'PC-AMO:    {int(pc_amo_yr.year[0])}–{int(pc_amo_yr.year[-1])}  ({len(pc_amo_yr)} ani)')
print(f'PC-Tripol: {int(pc_tripol_yr.year[0])}–{int(pc_tripol_yr.year[-1])}  ({len(pc_tripol_yr)} ani)')

## 5 — Figura EOF (Nature-style)

In [ ]:
fig = climpy.ClimPlot(
    nrows=2, ncols=2, w=climpy.NATURE_2COL, h=5.8,
    map_proj=(climpy.Map(), climpy.Map(), 'ts', 'ts'),
)
MAP_KW = dict(
    map_extent=[-80,25,-2,82],
    vmin=-0.5, vmax=0.5, cbar_step=0.25, nlevels=11,
    cbar_label='SST anomaly (°C)',
    cmap=CMAP_SST, show_land=True, coastlines_lw=0.35,
    lon_ticks=[-60,-30,0], lat_ticks=[0,30,60],
)
fig[0].map(eof_amo,    title=f'(a) EOF-AMO  ({float(frac[0]):.1f}%)',    **MAP_KW)
fig[1].map(eof_tripol, title=f'(b) EOF-Tripole  ({float(frac[1]):.1f}%)', **MAP_KW)
fig[2].ts(pc_amo_yr,    pc_amo_ma7_yr,
    title='(c) PC-AMO  [1854–2019]', ylabel='Amplitude (std)',
    labels=['Annual','7-yr MA'], colors=['#0077BB','#CC3311'])
fig[3].ts(pc_tripol_yr, pc_tripol_ma7_yr,
    title='(d) PC-Tripole  [1854–2019]', ylabel='Amplitude (std)',
    labels=['Annual','7-yr MA'], colors=['#0077BB','#CC3311'])
fig.savefig('figures/fig_eof.pdf')
fig.show()

## 6 — Corelații AMO & Tripole × SAT / SLP / Precip

**Indici anuali** (ne-neteziți) pentru corelații → N_eff ≈ 30–35 ani după corecție autocorelare.  
Cu MA 7 ani N_eff ≈ 5–8, practic niciodată semnificativ.  
**Stippling** = p < 0.05 după FDR Benjamini-Hochberg (Wilks 2006).

In [ ]:
def align_yr(pc, field):
    """Aliniază PC și câmp pe anii comuni."""
    common = sorted(set(pc['year'].values) & set(field['year'].values))
    return pc.sel(year=common), field.sel(year=common)

print('Calculez corelații cu indici ANUALI + FDR ...')

# SAT DJFM — indici anuali
s_a, sat_d   = align_yr(pc_amo_yr,    sat_djfm)
s_t, sat_d2  = align_yr(pc_tripol_yr, sat_djfm)
r_amo_sat,    p_amo_sat    = correlation_pvalue(s_a, sat_d,  dim='year', fdr=False)
r_tripol_sat, p_tripol_sat = correlation_pvalue(s_t, sat_d2, dim='year', fdr=False)
print(f'  ✓ SAT  (N={len(s_a)} ani)')

# SLP DJFM
s_a, slp_d   = align_yr(pc_amo_yr,    slp_djfm)
s_t, slp_d2  = align_yr(pc_tripol_yr, slp_djfm)
r_amo_slp,    p_amo_slp    = correlation_pvalue(s_a, slp_d,  dim='year', fdr=False)
r_tripol_slp, p_tripol_slp = correlation_pvalue(s_t, slp_d2, dim='year', fdr=False)
print(f'  ✓ SLP  (N={len(s_a)} ani)')

# Precip DJFM
s_a, pr_d    = align_yr(pc_amo_yr,    pr_djfm)
s_t, pr_d2   = align_yr(pc_tripol_yr, pr_djfm)
r_amo_pr,     p_amo_pr     = correlation_pvalue(s_a, pr_d,  dim='year', fdr=False)
r_tripol_pr,  p_tripol_pr  = correlation_pvalue(s_t, pr_d2, dim='year', fdr=False)
print(f'  ✓ Precip  (N={len(s_a)} ani)')
print('Gata.')

In [ ]:
# ── Figura corelații 3×2 cu stippling ───────────────────────────────────────
fig = climpy.ClimPlot(
    nrows=3, ncols=2, w=climpy.NATURE_2COL, h=8.5,
    map_proj=(climpy.Map(-30),)*6,
)
BASE = dict(
    vmin=-0.7, vmax=0.7, cbar_step=0.35, nlevels=15,
    cbar_label='Pearson r',
    show_land=False, show_ocean=False, coastlines_lw=0.35,
)
panels = [
    (r_amo_sat,    p_amo_sat,    '(a) AMO × SAT DJFM',        CMAP_SAT),
    (r_tripol_sat, p_tripol_sat, '(b) Tripole × SAT DJFM',    CMAP_SAT),
    (r_amo_slp,    p_amo_slp,    '(c) AMO × SLP DJFM',        CMAP_SLP),
    (r_tripol_slp, p_tripol_slp, '(d) Tripole × SLP DJFM',    CMAP_SLP),
    (r_amo_pr,     p_amo_pr,     '(e) AMO × Precip DJFM',     CMAP_PRECIP),
    (r_tripol_pr,  p_tripol_pr,  '(f) Tripole × Precip DJFM', CMAP_PRECIP),
]
for i, (r, p, title, cmap) in enumerate(panels):
    fig[i].map(r, title=title, cmap=cmap, **BASE)
    fig[i].stipple(p.values, r['lat'].values, r['lon'].values, pthresh=0.05, size=8, density=1)
    fig.axes[i].set_facecolor('#F2F0EB')

fig.fig.text(0.5, 0.01,
    'Indici: ANUALI (ne-neteziți) | Stippling: p<0.05 (corecție autocorelare AR1)',
    ha='center', fontsize=5.5, color='#888', style='italic')
fig.savefig('figures/fig_corr.pdf')
fig.show()

## 7 — Regresii AMO & Tripole × SAT / SLP / Precip

Layout **3×2** (identic cu corelațiile): col. 1 = AMO, col. 2 = Tripole.  
Corecție autocorelare AR(1) (Quenouille 1952) + FDR.  
Indici **anuali** (nu MA) pentru semnificație corectă.

In [ ]:
print('Calculez regresii (autocorelare + FDR) ...')

kw = dict(dim='year', normalise=True, autocorr_correction=True, fdr=False)

s_a, sat_d  = align_yr(pc_amo_yr,    sat_djfm)
s_t, sat_d2 = align_yr(pc_tripol_yr, sat_djfm)
sl_amo_sat,    p_sl_amo_sat    = regression_pvalue(s_a, sat_d,  **kw)
sl_tripol_sat, p_sl_tripol_sat = regression_pvalue(s_t, sat_d2, **kw)
print('  ✓ SAT')

s_a, slp_d  = align_yr(pc_amo_yr,    slp_djfm)
s_t, slp_d2 = align_yr(pc_tripol_yr, slp_djfm)
sl_amo_slp,    p_sl_amo_slp    = regression_pvalue(s_a, slp_d,  **kw)
sl_tripol_slp, p_sl_tripol_slp = regression_pvalue(s_t, slp_d2, **kw)
print('  ✓ SLP')

s_a, pr_d   = align_yr(pc_amo_yr,    pr_djfm)
s_t, pr_d2  = align_yr(pc_tripol_yr, pr_djfm)
sl_amo_pr,     p_sl_amo_pr     = regression_pvalue(s_a, pr_d,  **kw)
sl_tripol_pr,  p_sl_tripol_pr  = regression_pvalue(s_t, pr_d2, **kw)
print('  ✓ Precip')
print('Gata.')

In [ ]:
# ── Figura regresii 3×2 cu stippling ────────────────────────────────────────
fig = climpy.ClimPlot(
    nrows=3, ncols=2, w=climpy.NATURE_2COL, h=8.5,
    map_proj=(climpy.Map(-30),)*6,
)
reg_panels = [
    (sl_amo_sat,    p_sl_amo_sat,    '(a) AMO → SAT DJFM',       -0.6,  0.6,  0.3,  '°C / std(AMO)',   CMAP_SAT),
    (sl_tripol_sat, p_sl_tripol_sat, '(b) Tripole → SAT DJFM',   -0.6,  0.6,  0.3,  '°C / std(Tri)',   CMAP_SAT),
    (sl_amo_slp,    p_sl_amo_slp,    '(c) AMO → SLP DJFM',       None, None, None, 'hPa / std(AMO)',  CMAP_SLP),
    (sl_tripol_slp, p_sl_tripol_slp, '(d) Tripole → SLP DJFM',   None, None, None, 'hPa / std(Tri)',  CMAP_SLP),
    (sl_amo_pr,     p_sl_amo_pr,     '(e) AMO → Precip DJFM',    -0.3,  0.3,  0.15, 'mm/d / std(AMO)', CMAP_PRECIP),
    (sl_tripol_pr,  p_sl_tripol_pr,  '(f) Tripole → Precip DJFM',-0.3,  0.3,  0.15, 'mm/d / std(Tri)', CMAP_PRECIP),
]
for i, (slope, pval, title, vmin, vmax, step, label, cmap) in enumerate(reg_panels):
    # Dacă vmin/vmax sunt None → calculează din date
    if vmin is None:
        vm = float(np.nanpercentile(np.abs(slope.values), 97))
        vmin, vmax = -vm, vm
        step = round(vm / 2, 3) if vm < 0.5 else round(vm / 2, 2)
    fig[i].map(slope, title=title, cmap=cmap,
               vmin=vmin, vmax=vmax, cbar_step=step, nlevels=13,
               cbar_label=label,
               show_land=False, show_ocean=False, coastlines_lw=0.35)
    fig[i].stipple(pval.values, slope['lat'].values, slope['lon'].values, pthresh=0.05, size=8, density=1)
    fig.axes[i].set_facecolor('#F2F0EB')

fig.fig.text(0.5, 0.01,
    'Indici: ANUALI | Corecție AR(1) (Quenouille 1952) | Stippling: p<0.05',
    ha='center', fontsize=5.5, color='#888', style='italic')
fig.savefig('figures/fig_regr.pdf')
fig.show()

## 8 — CCA: SST × SLP (moduri coupled AMOC-NAO)

**Barnett & Preisendorfer (1987)**: EOF pre-filtrare → SVD cross-covarianță → test semnificație prin permutare.

In [ ]:
# SLP anual pe domeniu Atlantic-Europa
slp_ann_r = climpy.preprocess(
    DATA / 'slp.nc', var='slp',
    fill_value     = 9.96921e+36,   # ← fill value NCEP
    lon_convention = '[-180,180]',
    time=FIELD_PERIOD, ref_period=FIELD_REF,
    lat=(-20, 80), lon=(-90, 40),
    compute_annual=True, subtract_gm=False,
)
annual_slp = slp_ann_r['annual'].dropna('year', how='all')
# Conversie unități — aceeași logică ca la slp_djfm de mai sus
if float(annual_slp.mean()) > 10000:   # e în Pa
    annual_slp = annual_slp / 100.0
    print('SLP CCA: convertit Pa → hPa')
else:
    print('SLP CCA: deja în hPa, fără conversie')

common_yrs = sorted(set(annual_sst['year'].values) & set(annual_slp['year'].values))
sst_cca    = annual_sst.sel(year=common_yrs)
slp_cca    = annual_slp.sel(year=common_yrs)
print(f'SST CCA: {dict(sst_cca.sizes)}')
print(f'SLP CCA: {dict(slp_cca.sizes)}')
print(f'Perioadă: {common_yrs[0]}–{common_yrs[-1]}')

In [ ]:
cca = CCA(sst_cca, slp_cca, min_variance=70.0, lat_weights=True)
cca.summary()

print('\nTest semnificație (500 permutări) ...')
pvals_cca = cca.significance_test(n_perm=500, rng=np.random.default_rng(42))
print(f"{'Mod':>5}  {'r_canon':>8}  {'p-value':>8}  {'Sig':>5}")
print('-' * 32)
for i, (r, p) in enumerate(zip(cca.cancorrs, pvals_cca)):
    print(f"{i+1:>5}  {r:>8.3f}  {p:>8.3f}  {'★' if p<0.05 else '·' if p<0.10 else '':>5}")

In [ ]:
xpat  = cca.x_patterns()
ypat  = cca.y_patterns()
xts   = cca.x_timeseries()
yts   = cca.y_timeseries()
corrs = cca.canonical_correlations()

fig = climpy.ClimPlot(
    nrows=3, ncols=2, w=climpy.NATURE_2COL, h=8.0,
    map_proj=(
        climpy.Map(), climpy.Map(),
        climpy.Map(-30), climpy.Map(-30),
        'ts', 'ts',
    ),
)

def norm_pat(da):
    v = np.nanstd(da.values)
    return da / v if v > 0 else da

# Rândul 1: SST patterns
for mi, pi in enumerate([0, 1]):
    pat = norm_pat(xpat.isel(mode=mi))
    vm  = max(0.3, float(np.nanpercentile(np.abs(pat.values), 95)))
    sig = ' ★' if pvals_cca[mi] < 0.05 else ''
    fig[pi].map(pat,
        title=f'({"ab"[mi]}) CCA-{mi+1} SST  r={corrs[mi]:.2f}{sig}',
        map_extent=[-80,25,-2,82],
        vmin=-vm, vmax=vm, nlevels=11, cbar_step=round(vm/2, 2),
        cbar_label='Normalised amplitude',
        cmap=CMAP_SST, show_land=True, coastlines_lw=0.35,
        lon_ticks=[-60,-30,0], lat_ticks=[0,30,60],
    )

# Rândul 2: SLP patterns
for mi, pi in enumerate([2, 3]):
    pat = norm_pat(ypat.isel(mode=mi))
    vm  = max(0.5, float(np.nanpercentile(np.abs(pat.values), 95)))
    fig[pi].map(pat,
        title=f'({"cd"[mi]}) CCA-{mi+1} SLP',
        vmin=-vm, vmax=vm, nlevels=11, cbar_step=round(vm/2, 2),
        cbar_label='Normalised amplitude',
        cmap=CMAP_SLP, show_land=False, show_ocean=False, coastlines_lw=0.35,
    )
    fig.axes[pi].set_facecolor('#F2F0EB')

# Rândul 3: time series
for mi, pi, lbl in [(0, 4, '(e)'), (1, 5, '(f)')]:
    ts_x   = xts.isel(mode=mi)
    ts_y   = yts.isel(mode=mi)
    ts_xma = climpy.moving_average(ts_x, n=7)
    fig[pi].ts(ts_x, ts_y, ts_xma,
        title=f'{lbl} CCA-{mi+1}: SST & SLP time series',
        ylabel='Canonical amplitude',
        labels=['SST canonical','SLP canonical','SST 7-yr MA'],
        colors=['#0077BB','#CC3311','#0077BB'],
        linewidths=[0.7, 0.7, 2.0],
        alphas=[0.4, 0.5, 1.0],
        shade_first=False,
    )

fig.fig.text(0.5, 0.01,
    'CCA Barnett & Preisendorfer (1987) | EOF pre-filtrare 70% | '
    'Semnificație: permutare 500 iter. | ★ p<0.05',
    ha='center', fontsize=5.5, color='#888', style='italic')
fig.savefig('figures/fig_cca.pdf')
fig.show()

In [ ]:
# ── Descărcare figuri (Colab) ────────────────────────────────────────────────
try:
    from google.colab import files
    for f in ['figures/fig_eof.pdf','figures/fig_corr.pdf',
              'figures/fig_regr.pdf','figures/fig_cca.pdf']:
        files.download(f)
        print(f'  ↓ {f}')
except ImportError:
    print('Nu ești în Colab — figurile sunt în folderul figures/')